In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from pprint import pprint, pformat
import fitz  # Import the PyMuPDF library
import csv
import pandas as pd
from langchain import hub
from langchain_openai  import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import AIMessage, HumanMessage, SystemMessage, Document
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import tiktoken

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent, create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI, OpenAI

from nanonets import NANONETSOCR


load_dotenv(find_dotenv())

True

### Global variables

In [3]:
input_file = r"N:\Dev\AI\Underwriting\data\clients\Safia Seyed\Safia Seyed Paystub.pdf"
input_file = r"N:\Dev\AI\Underwriting\data\clients\Alex and Patricia\Alex\Alexander_Deaibes_Pay_Stub(s)_9tM4WQ8J9J3yc13DgjRkEo.pdf"
# Extracted tables:
extracted_csvs_path = os.path.join(os.path.dirname(input_file), os.path.splitext(os.path.basename(input_file))[0] + " extracted_tables")
os.makedirs(extracted_csvs_path, exist_ok=True)
nano_extracted_tables_csv_path = os.path.join(extracted_csvs_path, os.path.splitext(os.path.basename(input_file))[0] + "_extracted_tables.csv" )

# Nanonets OCR

In [3]:
model = NANONETSOCR()
model.set_token(os.getenv("NANONESTS_TABLE_MODEL_TOKEN"))

In [ ]:
model.convert_to_csv(input_file, output_file_name=nano_extracted_tables_csv_path)

# Parse CSV

In [4]:
def extract_tables_to_csv(csv_file_path, output_path):
    def save_table(table_lines, table_number, output_folder):
        output_file_path = os.path.join(output_folder, f"Table_{table_number}.csv")
    
        # Check if the first row is empty or contains only commas
        if not any(table_lines[0]) or all(cell == '' for cell in table_lines[0]):
            table_lines = table_lines[1:]  # Remove the first row if it's empty
    
        with open(output_file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(table_lines)
        print(f"Table {table_number} saved to {output_file_path}")
        return output_file_path  # Return the path to the saved file
    
    
    # Extract directory and base name without extension
    csv_directory = os.path.dirname(csv_file_path)
    csv_base_name = os.path.basename(csv_file_path)
    csv_name_without_ext = os.path.splitext(csv_base_name)[0]

    # Create output folder path
    output_folder = output_path#os.path.join(csv_directory, csv_name_without_ext)

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize variables
    current_table_number = 0
    current_table_lines = []
    table_started = False
    table_paths = []  # Initialize an empty list to store file paths

    # Read the CSV file
    with open(csv_file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        for row in reader:
            # Convert row list to string to check if it's a table header
            row_str = ','.join(row)
            if row_str.startswith('TABLE '):
                # If we're already in a table, save the current table
                if table_started:
                    path = save_table(current_table_lines, current_table_number, output_folder)
                    table_paths.append(path)  # Add the path to the list
                    current_table_lines = []  # Reset for the next table
                current_table_number += 1
                table_started = True  # Start a new table
            elif table_started:
                # Add row to current table
                current_table_lines.append(row)

    # Save the last table after file ends
    if table_started:
        save_table(current_table_lines, current_table_number, output_folder)
        table_paths.append(path)  # Add the path to the list
        
    return table_paths

def extract_tables_to_dfs(csv_file_path):
    # Initialize variables
    current_table_number = 0
    current_table_lines = []
    table_started = False
    dfs = []  # Initialize an empty list to store DataFrames

    # Read the CSV file
    with open(csv_file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        for row in reader:
            # Check if it's a table header
            row_str = ','.join(row)
            if row_str.startswith('TABLE '):
                # If we're already in a table, convert the current table to a DataFrame
                if table_started:
                    df = pd.DataFrame(current_table_lines[1:], columns=current_table_lines[0])
                    dfs.append(df)
                    current_table_lines = []  # Reset for the next table
                current_table_number += 1
                table_started = True  # Start a new table
            elif table_started:
                # Add row to current table
                current_table_lines.append(row)

    # Convert the last table to a DataFrame after file ends, if any
    if table_started:
        df = pd.DataFrame(current_table_lines[1:], columns=current_table_lines[0])
        dfs.append(df)

    return dfs

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ''
    for page in doc:
        text += page.get_text()
    doc.close()

    return text

In [5]:
# Example usage
# csv_file_path = 'data/pdfs/Yash Paystub Nov 17 2023_extracted.csv'
# csv_file_path = r'N:\Dev\AI\Underwriting\output3.csv'
table_paths = extract_tables_to_csv(nano_extracted_tables_csv_path, extracted_csvs_path)
table_dfs = extract_tables_to_dfs(nano_extracted_tables_csv_path)
table_text = extract_text_from_pdf(input_file)

Table 1 saved to N:\Dev\AI\Underwriting\data\clients\Alex and Patricia\Alex\Alexander_Deaibes_Pay_Stub(s)_9tM4WQ8J9J3yc13DgjRkEo extracted_tables\Table_1.csv
Table 2 saved to N:\Dev\AI\Underwriting\data\clients\Alex and Patricia\Alex\Alexander_Deaibes_Pay_Stub(s)_9tM4WQ8J9J3yc13DgjRkEo extracted_tables\Table_2.csv
Table 3 saved to N:\Dev\AI\Underwriting\data\clients\Alex and Patricia\Alex\Alexander_Deaibes_Pay_Stub(s)_9tM4WQ8J9J3yc13DgjRkEo extracted_tables\Table_3.csv
Table 4 saved to N:\Dev\AI\Underwriting\data\clients\Alex and Patricia\Alex\Alexander_Deaibes_Pay_Stub(s)_9tM4WQ8J9J3yc13DgjRkEo extracted_tables\Table_4.csv


In [6]:
# print(table_paths)

In [7]:
# table_dfs[0]

In [21]:
model = "gpt-4-turbo-preview"

dfs_chat_agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model=model), 
    table_dfs, 
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

csv_chat_agent = create_csv_agent(
    ChatOpenAI(temperature=0, model=model),
    table_paths,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

text_chat = ChatOpenAI(temperature=0, model=model)
text_prompt = f'''
The text below is extracted from financial payslips, answer the question based on the text provided.
Text:
{table_text}

Question:

'''

### Table post-processing

In [ ]:
post_processor = ChatOpenAI(temperature=0, model=model)

post_processor_prompt = "Given a CSV file, table that has been extracted using OCR. The headers of the CSV file might not be accurate to the data of the actual table. Reformat the table, and remove unnecessary headers only if they are unnecessary to match the desired table and return ONLY the revised CSV table\nCSV contents:\n"

# for csv_table in table_paths:
with open(table_paths[0], 'r') as file:
    raw_csv = file.read()

print(raw_csv)

In [27]:
print(post_processor.invoke(str(post_processor_prompt) + str(raw_csv)).content)


NameError: name 'post_processor' is not defined

## Some qureying

In [24]:
question = '''You are provided with financial pay stubs. What is the Regular Salary? Provide the lower value between Regular and Gross Salary if applicable. Terminology might differ. Regular earnings refer to earnings without any bonuses or benefits

In the final response provide ONLY the numerical value of the Regular earnings.
'''

In [25]:
csv_chat_agent.run(question)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df1.head()'}`


   EARNINGS  DATE\nYMMDD    RATE CURRENT\nHRS/UNITS CURRENT\nAMOUNT  \
0   REGULAR          NaN   0.000              72.50         2129.98   
1    ADJUST          NaN   0.000               0.00            0.00   
2  LEAWOPAY          NaN  29.379              21.75          638.99   
3   ADJ REG          NaN   0.000             21.75-         638.99-   
4  UNCL BEN          NaN   0.000               0.00           89.46   

  YTD\nHRS/UNITS YTD\nAMOUNT  
0           0.00    39830.82  
1         377.00    1672.05-  
2          65.25     1916.98  
3         65.25-    1916.98-  
4           0.00     2174.54  2129.98

> Finished chain.


'2129.98'

In [26]:
dfs_chat_agent.run(question)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df1.loc[df1[0] == 'REGULAR', 'CURRENT AMOUNT'].values[0]"}`


KeyError: 0
Invoking: `python_repl_ast` with `{'query': "df1.loc[df1.iloc[:, 0] == 'REGULAR', 'CURRENT AMOUNT'].values[0]"}`


KeyError: 'CURRENT AMOUNT'
Invoking: `python_repl_ast` with `{'query': "df1.loc[df1.iloc[:, 0] == 'REGULAR', 'CURRENT AMOUNT'].values"}`


KeyError: 'CURRENT AMOUNT'
Invoking: `python_repl_ast` with `{'query': "df1.loc[df1.iloc[:, 0] == 'REGULAR', df1.columns[5]].values[0]"}`


['REGULAR' '' '0.0000' '72.50' '2129.98' '0.00' '39830.82']The Regular earnings are **2129.98**.

> Finished chain.


'The Regular earnings are **2129.98**.'

In [23]:
type(text_prompt)
text_chat.invoke(text_prompt + question)

AIMessage(content='39830.82', response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 1060, 'total_tokens': 1064}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': 'fp_a7daf7c51e', 'finish_reason': 'stop', 'logprobs': None})

In [14]:
(csv_chat_agent.agent.dict())

{'runnable': {'name': None,
  'first': {'name': None,
   'mapper': {'name': None,
    'steps': {'agent_scratchpad': RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))}}},
  'middle': [{'name': None,
    'input_variables': ['agent_scratchpad', 'input'],
    'input_types': {'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]},
    'output_parser': None,
    'partial_variables': {},
    'metadata': None,
    'tags': None,
    'messages': [{'content': '\nYou are working with 4 pandas dataframes in Python named df1, df2, etc.\nThis is the result of `print(df.head())` for each dataframe:\n|    | EARNINGS   |    DATE |   RATE | CURRENT     | CURRENT   | YTD         | YTD      |\n|    |            |   YMMDD 

## JSON output parser attempt

In [10]:
# Chain Attempt
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define your desired data structure.
class Salary(BaseModel):
    salary: str = Field(description="The salary of the client")

question="Please extract the gross pay. If there is a difference between the Gross and Regular earnings, provide the regular earnings. Make sure you are not providing the hourly rate"
model = ChatOpenAI(temperature=0)
parser = JsonOutputParser(pydantic_object=Salary)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | csv_chat_agent | model | parser

json_str = chain.invoke({"query": question})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df1['CURRENT AMOUNT'].sum()"}`


KeyError: 'CURRENT AMOUNT'
Invoking: `python_repl_ast` with `{'query': "df1['CURRENT AMOUNT'].sum()"}`


KeyError: 'CURRENT AMOUNT'
Invoking: `python_repl_ast` with `{'query': 'df1.columns'}`


Index(['EARNINGS', 'DATE\nYMMDD', 'RATE', 'CURRENT\nHRS/UNITS',
       'CURRENT\nAMOUNT', 'YTD\nHRS/UNITS', 'YTD\nAMOUNT'],
      dtype='object')
Invoking: `python_repl_ast` with `{'query': "df1['CURRENT\\nAMOUNT'].sum()"}`


TypeError: can only concatenate str (not "int") to str
Invoking: `python_repl_ast` with `{'query': "df1['CURRENT\\nAMOUNT'].astype(str).str.replace('-', '').astype(float).sum()"}`


8329.21```json
{
  "salary": "8329.21"
}
```

> Finished chain.


ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

# Custom agents

In [ ]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

## Custom Tools

In [ ]:
from langsmith import Client
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = f"Underwriting project"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
client = Client()

In [ ]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [ ]:
llm = ChatOpenAI(temperature=0, model=model)

from langchain.prompts import PromptTemplate

text_searcher_prompt = PromptTemplate.from_template('''Search the following text which was extracted from a PDF document and gather the information requested by the query. The text might be structured oddly so deductive reasoning might be needed to get the relevant information.  

Query:
{query}

Text:
{text}
'''
)

In [ ]:
from typing import List


# Add error handling

@tool
def text_searcher(query: str) -> str:
    """Given a query the text_searcher tool will search the text to gather the information requested in the query"""
    
    return llm.invoke(text_searcher_prompt.format(query=query, text=table_text))

@tool
def csv_searcher(query: str) -> str:
    """Given a query the csv_searcher tool will use the csv_chat_agent to search the structured tables and gather the information requested in the query"""
    
    return csv_chat_agent.run(query)

tools = [csv_searcher, ]#text_searcher]

In [ ]:
from langchain_core.prompts import SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, \
    ChatPromptTemplate
from pprint import pprint
agent_system_message = ''' You are an underwriting assistant who needs to extract and gather information from the provided documents. You must try to gather information from the structured CSV documents'''
 # and varify them by searching the text directly. Some data might not be contained in the CSV files and should be gathered from the text.  
# '''

agent_messages = [
    # SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=agent_system_message)),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')
]

temp = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=agent_system_message)),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

temp
# 
# [temp.append(message) for message in agent_messages]
# temp

# from langchain import hub
# 
# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages[0].prompt

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
question = "Please extract the following data if available: Gross pay, regular pay, YTD payments, hours and hourly rate (if applicable), payslip pay period (weekly, bi weekly, semi monthly, monthly), payment start and end dates"
question="Please extract the gross pay. If there is a difference between the Gross and Regular earnings, provide the regular earnings. Make sure you are not providing the hourly rate"
l = list(agent_executor.stream(
        {
            "input":question,
        }
    )
)

In [ ]:
pprint(l)